Using the Vecalign script vecalign.py (partially modified) from https://github.com/thompsonb/vecalign, the code takes as input the originary files containing the sentences, the overlap files, the files containing the embeddings. The output of the alignment is a list of indexes pairs and a float number with the compute of the alignment cost.

The modification of the vecalign.py file consists in generating a .TXT file containing the output of the alignment.

In [ ]:
# Use latest conda
!conda update conda -y
# Create conda environment
!conda create  --force -y --name vecalign python=3.7
# Activate new environment
!source `conda info --base`/etc/profile.d/conda.sh # See: https://github.com/conda/conda/issues/7980
!conda activate vecalign
# Install required packages
!conda install -y -c anaconda cython
!conda install -y -c anaconda numpy
!pip install mcerp 

In [ ]:
## generate overlaps file using the eng and lat sentences in the sentences_NH folder
!python overlap.py -i sentences_NH/engbook37 -o sentences_NH/engoverlaps37.eng -n 10
!python overlap.py -i sentences_NH/latbook37 -o sentences_NH/latoverlaps37.lat -n 10

In [ ]:
pip install sentence_transformers

In [ ]:
## import the S-BERT Python library sentence_transformer
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
for i in range(1,38):    
    print(i)
    
    ## get the eng overlap file
    path = "/Users/u0154817/OneDrive - KU Leuven/Desktop/sentences_NH/engoverlaps"+str(i)+'.eng'
    with open(path, "r", encoding="utf8") as engfile:
        overlap_sentences_eng = []
        for line in engfile: ## create a list of the overlap eng sentences
            overlap_sentences_eng.append(line)
    
    ## get the lat overlap file
    path = "/Users/u0154817/OneDrive - KU Leuven/Desktop/sentences_NH/latoverlaps"+str(i)+'.lat'
    with open(path, "r", encoding="utf8") as latfile:
        overlap_sentences_lat = []
        for line in latfile:  ## create a list of the overlap lat sentences
            overlap_sentences_lat.append(line)
    
    ## generate the embeddings of the eng and lat overlap sentences
    overlaps_embeddings_eng_LaBSE = model.encode(overlap_sentences_eng)
    overlaps_embeddings_lat_LaBSE = model.encode(overlap_sentences_lat)
    
    overlaps_embeddings_eng_LaBSE.tofile('overlaps.eng.'+str(i)+'.LaBSE.emb')
    overlaps_embeddings_lat_LaBSE.tofile('overlaps.lat.'+str(i)+'.LaBSE.emb')

In [ ]:
## calculate the sentence similarity using the sentence files, the overlap files, and the sentence embeddings
!python vecalign.py --alignment_max_size 8 --src sentences_NH/engbook37 --tgt sentences_NH/latbook37 \
   --src_embed sentences_NH/engoverlaps37.eng sentences_NH/overlaps.eng.37.LaBSE.emb  \
   --tgt_embed sentences_NH/latoverlaps37.lat sentences_NH/overlaps.lat.37.LaBSE.emb